# Notebook Lecture 9: Frequency Response
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces frequency response analysis methods for linear time-invariant systems, covering Bode plots, polar plots, and their applications in control system design.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

### 1. Master Bode Plot Construction
- Recognize why point-by-point frequency analysis is impractical
- Know and apply the rules for sketching Bode plots
- Understand the significance of corner frequencies and asymptotic behavior

### 2. Apply Construction Rules to Basic Elements
- Draw Bode plots for:
  - Static gains
  - Integrators and differentiators
  - First-order systems
  - Second-order systems
  - Non-minimum phase zeros


## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output
from IPython.display import display, clear_output
from scipy import signal
import cmath

# Frequency Response Recap

## Key Concept: System Response to Sinusoidal Inputs

Linear Time-Invariant (LTI) systems have a fundamental property: when excited by a sinusoidal input, their output is also sinusoidal with:
- Same frequency as the input
- Modified amplitude
- Shifted phase

A sinusoidal input can be written as:
$$u(t) = e^{j\omega t} + e^{-j\omega t} = 2\cos(\omega t)$$

For a system with transfer function $G(s)$, the output is:
$$y(t) = G(j\omega)e^{j\omega t} + G(-j\omega)e^{-j\omega t}$$

Using Euler's formula, this simplifies to:
$$y(t) = |G(j\omega)|\cos(\omega t + \angle G(j\omega))$$

### Key Components
1. **Magnitude Response**: $|G(j\omega)|$
   - Ratio of output amplitude to input amplitude
   - How much the system amplifies/attenuates the input

2. **Phase Response**: $\angle G(j\omega)$
   - Time shift between input and output
   - How much the system delays/advances the signal

As you might expect, the magnitude and phase responses are functions of the input frequency $\omega$. The frequency response of a system is the collection of these responses for all possible input frequencies. Therefore, it is very useful to plot the magnitude and phase responses as functions of frequency, as it will give us a good understanding of how the system behaves for different input frequencies and also give us a good insight into the stability of the system.

The first intuitive thinking on how to plot the frequency response of a system is to evaluate the transfer function at different frequencies and plot the magnitude and phase of the transfer function. 

In [ ]:
# Initialize output widget for plotting
output = widgets.Output()

# Global variables to store points
frequencies = []
magnitudes = []
phases = []

def parse_polynomial(poly_str):
    """Convert string polynomial coefficients to list of floats"""
    return [float(coef.strip()) for coef in poly_str.split()]

def calculate_response(freq, num_str, den_str):
    """Calculate magnitude and phase at given frequency"""
    try:
        num = parse_polynomial(num_str)
        den = parse_polynomial(den_str)
        sys = signal.TransferFunction(num, den)
        w, mag, phase = signal.bode(sys, w=[freq])
        return mag[0], phase[0]
    except Exception as e:
        print(f"Error calculating response: {str(e)}")
        return 0, 0

def update_plots(frequency, num_str, den_str, show_full=False):
    with output:
        clear_output(wait=True)
        
        try:
            # Add new point if not already present
            if frequency not in frequencies:
                mag, phase = calculate_response(frequency, num_str, den_str)
                frequencies.append(frequency)
                magnitudes.append(mag)
                phases.append(phase)
                
                # Sort points by frequency
                sorted_indices = np.argsort(frequencies)
                frequencies[:] = [frequencies[i] for i in sorted_indices]
                magnitudes[:] = [magnitudes[i] for i in sorted_indices]
                phases[:] = [phases[i] for i in sorted_indices]
            
            # Create figure with side-by-side plots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))
            
            # Plot measured points with lines
            if frequencies:
                ax1.semilogx(frequencies, magnitudes, 'b.-', label='Measured Response')
                ax2.semilogx(frequencies, phases, 'r.-')
                # Highlight the current frequency point
                if frequency in frequencies:
                    idx = frequencies.index(frequency)
                else:
                    idx = -1
                ax1.semilogx(frequency, magnitudes[idx], 'go', markersize=10, label='Current Frequency')
                ax2.semilogx(frequency, phases[idx], 'go', markersize=10)
            
            # Show complete Bode plot if requested
            if show_full:
                w = np.logspace(-1, 3, 10000)
                sys = signal.TransferFunction(
                    parse_polynomial(num_str),
                    parse_polynomial(den_str)
                )
                w, mag, phase = signal.bode(sys, w=w)
                
                ax1.semilogx(w, mag, 'b--', alpha=0.5, label='Complete Response', linewidth=3)
                ax2.semilogx(w, phase, 'b--', alpha=0.5, linewidth=3)
            
            # Configure magnitude plot
            ax1.grid(True)
            ax1.set_ylabel('Magnitude (dB)')
            ax1.set_xlabel('Frequency (rad/s)')
            ax1.set_title('Magnitude Response')
            if frequencies:
                ax1.legend()
            
            # Configure phase plot
            ax2.grid(True)
            ax2.set_ylabel('Phase (degrees)')
            ax2.set_xlabel('Frequency (rad/s)')
            ax2.set_title('Phase Response')
            
            # Set axis limits
            ax1.set_xlim([0.1, 1000])
            ax2.set_xlim([0.1, 1000])
            ax1.set_ylim([-60, 20])
            ax2.set_ylim([-180, 180])
            
            plt.tight_layout()
            plt.show()
            
            # Print current transfer function and frequency
            print(f"\nTransfer Function:")
            print(f"Numerator coefficients: {num_str}")
            print(f"Denominator coefficients: {den_str}")
            print(f"Current frequency: ω = {frequency:.2f} rad/s")
            mag, phase = calculate_response(frequency, num_str, den_str)
            print(f"Magnitude: {mag:.1f} dB")
            print(f"Phase: {phase:.1f}°")
                
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
            print("\nPlease check your transfer function coefficients format.")
            print("Example format: '1 2 3' for 1s² + 2s + 3")

def clear_points(b):
    """Clear all points"""
    global frequencies, magnitudes, phases
    frequencies = []
    magnitudes = []
    phases = []
    update_plots(freq_slider.value, num_input.value, den_input.value)

def show_full(b):
    """Show the complete Bode plot"""
    update_plots(freq_slider.value, num_input.value, den_input.value, show_full=True)

# Create widgets
freq_slider = widgets.FloatLogSlider(
    value=1.0,
    base=10,
    min=-1,
    max=3,
    step=0.001,  # Smaller step size for smoother movement
    description='Frequency (rad/s):',
    style={'description_width': 'initial'},
    continuous_update=True
)

num_input = widgets.Text(
    value='1',
    description='Numerator:',
    style={'description_width': 'initial'},
    tooltip='Enter coefficients separated by spaces (highest power first)'
)

den_input = widgets.Text(
    value='1 1',
    description='Denominator:',
    style={'description_width': 'initial'},
    tooltip='Enter coefficients separated by spaces (highest power first)'
)

clear_button = widgets.Button(description='Clear Points')
show_full_button = widgets.Button(description='Show Full Response')

# Set up button callbacks
clear_button.on_click(clear_points)
show_full_button.on_click(show_full)

# Create layout
controls = widgets.VBox([
    widgets.HTML(value="<b>Enter transfer function coefficients:</b><br>Example: '1 2 3' for s² + 2s + 3"),
    num_input,
    den_input,
    freq_slider,
    widgets.HBox([clear_button, show_full_button])
])

# Connect the updates to widget changes
interactive_plot = interactive_output(
    update_plots, 
    {
        'frequency': freq_slider,
        'num_str': num_input,
        'den_str': den_input
    }
)

# Display everything
display(widgets.HBox([controls, output]))

# Initial plot
update_plots(freq_slider.value, num_input.value, den_input.value)

As demonstrated by the interactive tool above, we can compute the frequency response of a system numerically by:
1. Taking a transfer function
2. Evaluating it at different frequencies
3. Computing magnitude and phase at each frequency

While this method is accurate, it is not practical for manual analysis. Instead, we use Bode plots, which provide a systematic way to sketch frequency responses using simple rules. Let's look at an intuitive example to see what a Bode Plot rep

# Bode Plots

Bode plots provide a systematic way to sketch frequency responses using simple rules. These rules exploit the fact that:
1. Most transfer functions can be factored into basic elements
2. On a log-scale, multiplication becomes addition
3. Many terms can be approximated with straight lines

## Basic Elements and Their Rules

A transfer function can typically be broken down into:

1. **Static Gains**: $K$
   - Magnitude: Constant at $20\log_{10}|K|$ dB
   - Phase: 0° or 180° (if K < 0)

2. **Poles at Origin**: $\frac{1}{s^n}$
   - Magnitude: -20n dB/decade
   - Phase: -90n°

3. **Zeros at Origin**: $s^n$
   - Magnitude: +20n dB/decade
   - Phase: +90n°

4. **Real Poles**: $\frac{1}{\tau s + 1}$
   - Corner frequency at $\omega = \frac{1}{\tau}$
   - Magnitude:
     * Before corner: 0 dB/decade
     * After corner: -20 dB/decade
   - Phase:
     * Before corner: 0°
     * At corner: -45°
     * After corner: -90°

5. **Real Zeros**: $\tau s + 1$
   - Corner frequency at $\omega = \frac{1}{\tau}$
   - Magnitude:
     * Before corner: 0 dB/decade
     * After corner: +20 dB/decade
   - Phase:
     * Before corner: 0°
     * At corner: +45°
     * After corner: +90°


## Where do these rules come from?

Let's start with the static gain $K$. Below, you can see the point $H(s) = K$ plotted on the imaginary plane and next to it the corresponding Bode plot. As you change K, the magnitude, which is the distance to the origin changes, and you can observe the effect on the magnitude Bode plot. The phase Bode plot only changes when we pass the point where $K=0$ and we go from positive to negative and vice-versa, getting a phase of $0\degree$ or $180\degree$ respectively.


In [ ]:
def plot_constant_gain_with_imaginary_plane(K):
    # Bode Plot values
    magnitude = 20 * np.log10(abs(K)) * np.ones_like(frequencies)  # Constant dB gain
    phase = 180 if K < 0 else 0  # Phase is 180° if K is negative, 0° if positive

    # Complex plane representation
    real_part = K
    imag_part = 0

    # Create figure and axes (2 rows for Bode plot, 1 column for imaginary plane)
    fig = plt.figure(figsize=(12, 8))

    # Magnitude plot (Bode)
    ax_mag = fig.add_subplot(3, 2, 1)
    ax_mag.semilogx(frequencies, magnitude, label=f'|K| = {abs(K)}')
    ax_mag.set_title('Bode Plot - Magnitude')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True)
    ax_mag.legend(loc="upper left")
    ax_mag.set_ylim(-10, 40)  # Fixed Y-axis limits for magnitude plot

    # Phase plot (Bode)
    ax_phase = fig.add_subplot(3, 2, 3)
    phase_array = phase * np.ones_like(frequencies)
    ax_phase.semilogx(frequencies, phase_array, label=f'Phase = {phase}°')
    ax_phase.set_title('Bode Plot - Phase')
    ax_phase.set_ylabel('Phase (degrees)')
    ax_phase.set_xlabel('Frequency (rad/s)')
    ax_phase.grid(True)
    ax_phase.legend()
    ax_phase.set_ylim(-200, 200)  # Fixed Y-axis limits for phase plot

    # Imaginary Plane plot
    ax_imag = fig.add_subplot(3, 2, (2, 4))
    ax_imag.plot([0], [0], 'go')  # Plot origin with a distinct color
    ax_imag.plot([real_part], [imag_part], 'bo')  # Plot point for K
    ax_imag.plot([0, real_part], [0, imag_part], 'c-', lw=1.5, label='Magnitude')  # Line for magnitude in a different color

    # Set equal scaling for x and y axes
    max_val = 100  # Fixed axis scale for visualization
    ax_imag.set_xlim(-max_val, max_val)
    ax_imag.set_ylim(-max_val, max_val)
    ax_imag.set_aspect('equal', 'box')  # Ensures 1:1 aspect ratio
    ax_imag.axhline(0, color='grey', lw=0.5)
    ax_imag.axvline(0, color='grey', lw=0.5)
    ax_imag.set_title('Imaginary Plane - Constant Gain K')
    ax_imag.set_xlabel('Real')
    ax_imag.set_ylabel('Imaginary')
    ax_imag.grid(True)

    # Draw double-headed arrow along the magnitude line to indicate distance
    ax_imag.annotate('', xy=(real_part, imag_part), xytext=(0, 0), 
                     arrowprops=dict(arrowstyle='<->', color='c', lw=1.5))

    # Draw phase angle arc with an arrow at the end
    angle = np.deg2rad(phase)
    arc_radius = abs(K) * 0.3 if K != 0 else 10
    arc_x = [arc_radius * np.cos(t) for t in np.linspace(0, angle, 100)]
    arc_y = [arc_radius * np.sin(t) for t in np.linspace(0, angle, 100)]
    ax_imag.plot(arc_x, arc_y, 'r-', lw=1.5, label='Phase')
    ax_imag.arrow(arc_x[-2], arc_y[-2], arc_x[-1] - arc_x[-2], arc_y[-1] - arc_y[-2], 
                  shape='full', lw=0, length_includes_head=True, head_width=2, head_length=2, color='r')

    # Add a legend to the imaginary plane
    ax_imag.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

# Interactive widget for Constant Gain, allowing negative values
K_slider = widgets.FloatSlider(value=10, min=-100, max=100, step=1, description='K:')
interactive_constant_gain = interactive_output(plot_constant_gain_with_imaginary_plane, {'K': K_slider})
display(K_slider, interactive_constant_gain)

Now to the second rule, 

In [ ]:
# Frequency range for Bode plot
frequencies = np.logspace(-2, 2, 500)  # Frequency range for Bode plots

def plot_pole_at_origin(frequency):
    # Bode Plot values for a pole at the origin
    magnitude = -20 * np.log10(frequencies)  # Slope of -20 dB/decade
    phase = -90  # Constant phase of -90 degrees

    # Calculate magnitude and phase at the selected frequency
    selected_magnitude = -20 * np.log10(frequency)
    selected_phase = -90  # Phase remains -90 degrees for all frequencies

    # Convert frequency to complex plane point for the imaginary plot
    real_part = 0
    imag_part = -frequency

    # Create figure and axes (2 rows for Bode plot, 1 column for imaginary plane)
    fig = plt.figure(figsize=(12, 8))

    # Magnitude plot (Bode)
    ax_mag = fig.add_subplot(3, 2, 1)
    ax_mag.semilogx(frequencies, magnitude, label='Magnitude Slope -20 dB/decade')
    ax_mag.plot(frequency, selected_magnitude, 'ro')  # Highlight point at selected frequency
    ax_mag.set_title('Bode Plot - Magnitude (Pole at Origin)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True)
    ax_mag.legend(loc="upper left")
    ax_mag.set_ylim(-50, 50)  # Expanded Y-axis limits to fully display the slope

    # Phase plot (Bode)
    ax_phase = fig.add_subplot(3, 2, 3)
    phase_array = phase * np.ones_like(frequencies)
    ax_phase.semilogx(frequencies, phase_array, label=f'Phase = {phase}°')
    ax_phase.plot(frequency, selected_phase, 'ro')  # Highlight point at selected frequency
    ax_phase.set_title('Bode Plot - Phase (Pole at Origin)')
    ax_phase.set_ylabel('Phase (degrees)')
    ax_phase.set_xlabel('Frequency (rad/s)')
    ax_phase.grid(True)
    ax_phase.legend()
    ax_phase.set_ylim(-180, 0)  # Fixed Y-axis limits for phase plot

    # Imaginary Plane plot
    ax_imag = fig.add_subplot(3, 2, (2, 4))
    ax_imag.plot([0], [0], 'go')  # Plot origin with a distinct color
    ax_imag.plot([real_part, real_part], [0, imag_part], 'c-', lw=1.5, label='Magnitude')  # Line in downward direction
    ax_imag.plot(real_part, imag_part, 'ro')  # Point for selected frequency

    # Set equal scaling for x and y axes
    max_val = 1.5  # Fixed axis scale for visualization
    ax_imag.set_xlim(-max_val, max_val)
    ax_imag.set_ylim(-max_val, max_val)
    ax_imag.set_aspect('equal', 'box')  # Ensures 1:1 aspect ratio
    ax_imag.axhline(0, color='grey', lw=0.5)
    ax_imag.axvline(0, color='grey', lw=0.5)
    ax_imag.set_title('Imaginary Plane - Pole at Origin')
    ax_imag.set_xlabel('Real')
    ax_imag.set_ylabel('Imaginary')
    ax_imag.grid(True)

    # Draw double-headed arrow along the imaginary axis to indicate phase direction
    ax_imag.annotate('', xy=(0, imag_part), xytext=(0, 0), 
                     arrowprops=dict(arrowstyle='<->', color='c', lw=1.5))

    # Draw phase angle arc with an arrow at the end
    angle = np.deg2rad(-90)
    arc_radius = 0.3
    arc_x = [arc_radius * np.cos(t) for t in np.linspace(0, angle, 100)]
    arc_y = [arc_radius * np.sin(t) for t in np.linspace(0, angle, 100)]
    ax_imag.plot(arc_x, arc_y, 'r-', lw=1.5, label='Phase')
    ax_imag.arrow(arc_x[-2], arc_y[-2], arc_x[-1] - arc_x[-2], arc_y[-1] - arc_y[-2], 
                  shape='full', lw=0, length_includes_head=True, head_width=0.05, head_length=0.05, color='r')

    # Add a legend to the imaginary plane
    ax_imag.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

# Interactive slider for frequency selection
frequency_slider = widgets.FloatLogSlider(value=1, base=10, min=-2, max=2, step=0.01, description='Frequency (rad/s):')
interactive_plot = interactive_output(plot_pole_at_origin, {'frequency': frequency_slider})
display(frequency_slider, interactive_plot)

In [ ]:
# Frequency range for Bode plot
frequencies = np.logspace(-2, 2, 500)  # Frequency range for Bode plots

def plot_real_pole(frequency, corner_frequency=1):
    # Bode Plot values for a real pole
    magnitude = -20 * np.log10(np.maximum(frequencies, corner_frequency))  # Slope starts after corner frequency
    phase = -np.arctan2(frequencies, corner_frequency) * (180 / np.pi)  # Phase transition from 0° to -90°

    # Calculate magnitude and phase at the selected frequency
    selected_magnitude = -20 * np.log10(max(frequency, corner_frequency))
    selected_phase = -np.arctan2(frequency, corner_frequency) * (180 / np.pi)

    # Convert frequency to complex plane point for the imaginary plot
    real_part = -frequency / (frequency**2 + corner_frequency**2)**0.5
    imag_part = -corner_frequency / (frequency**2 + corner_frequency**2)**0.5

    # Create figure and axes (2 rows for Bode plot, 1 column for imaginary plane)
    fig = plt.figure(figsize=(12, 8))

    # Magnitude plot (Bode)
    ax_mag = fig.add_subplot(3, 2, 1)
    ax_mag.semilogx(frequencies, magnitude, label=f'Corner Frequency = {corner_frequency} rad/s')
    ax_mag.plot(frequency, selected_magnitude, 'ro')  # Highlight point at selected frequency
    ax_mag.set_title('Bode Plot - Magnitude (Real Pole)')
    ax_mag.set_ylabel('Magnitude (dB)')
    ax_mag.grid(True)
    ax_mag.legend(loc="upper left")
    ax_mag.set_ylim(-50, 10)  # Expanded Y-axis limits to fully display the slope

    # Phase plot (Bode)
    ax_phase = fig.add_subplot(3, 2, 3)
    ax_phase.semilogx(frequencies, phase, label=f'Corner Frequency = {corner_frequency} rad/s')
    ax_phase.plot(frequency, selected_phase, 'ro')  # Highlight point at selected frequency
    ax_phase.set_title('Bode Plot - Phase (Real Pole)')
    ax_phase.set_ylabel('Phase (degrees)')
    ax_phase.set_xlabel('Frequency (rad/s)')
    ax_phase.grid(True)
    ax_phase.legend()
    ax_phase.set_ylim(-180, 10)  # Fixed Y-axis limits for phase plot

    # Imaginary Plane plot
    ax_imag = fig.add_subplot(3, 2, (2, 4))
    ax_imag.plot([0], [0], 'go')  # Plot origin with a distinct color
    ax_imag.plot([0, real_part], [0, imag_part], 'c-', lw=1.5, label='Magnitude')  # Line from origin to point
    ax_imag.plot(real_part, imag_part, 'ro')  # Point for selected frequency

    # Set equal scaling for x and y axes
    max_val = 1.5  # Fixed axis scale for visualization
    ax_imag.set_xlim(-max_val, max_val)
    ax_imag.set_ylim(-max_val, max_val)
    ax_imag.set_aspect('equal', 'box')  # Ensures 1:1 aspect ratio
    ax_imag.axhline(0, color='grey', lw=0.5)
    ax_imag.axvline(0, color='grey', lw=0.5)
    ax_imag.set_title('Imaginary Plane - Real Pole')
    ax_imag.set_xlabel('Real')
    ax_imag.set_ylabel('Imaginary')
    ax_imag.grid(True)

    # Draw arrow indicating the phase angle on the imaginary plane
    ax_imag.annotate('', xy=(real_part, imag_part), xytext=(0, 0), 
                     arrowprops=dict(arrowstyle='<-', color='c', lw=1.5))

    # Draw phase angle arc with an arrow at the end
    angle = np.deg2rad(selected_phase) - np.pi/2
    arc_radius = 0.3
    arc_x = [arc_radius * np.cos(t) for t in np.linspace(0, angle, 100)]
    arc_y = [arc_radius * np.sin(t) for t in np.linspace(0, angle, 100)]
    ax_imag.plot(arc_x, arc_y, 'r-', lw=1.5, label='Phase')
    ax_imag.arrow(arc_x[-2], arc_y[-2], arc_x[-1] - arc_x[-2], arc_y[-1] - arc_y[-2], 
                  shape='full', lw=0, length_includes_head=True, head_width=0.05, head_length=0.05, color='r')

    # Add a legend to the imaginary plane
    ax_imag.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

# Interactive slider for frequency selection
frequency_slider = widgets.FloatLogSlider(value=1, base=10, min=-2, max=2, step=0.01, description='Frequency (rad/s):')
interactive_plot = interactive_output(plot_real_pole, {'frequency': frequency_slider})
display(frequency_slider, interactive_plot)


# Example

To get a bit more intuition on what the Bode plot actually shows us, let's look at a real-life example. Imagine a mass $m$ on wheels connected with a spring with spring stiffness $k$ to a handle that we can move horizontally. What happens in we move the handle back and forth in a sinusoidal fashion and slowly increase the frequency?

<iframe src="https://www.sccs.swarthmore.edu/users/12/abiele1/Linear/examples/freq.html" width="800" height="600"></iframe>

You can see that the frequency of the position of the mass stays the same, but it has a different magnitude and phase most of the time. This is exactly what the Bode plot shows us! Let's plot it.

The transfer function of the system is given by (with $m=1$, $b=0.5$, $k=1.6$, $u$ = input to system, $y$ = output (the position of the mass)):

$$
H(s) = \frac{Y(s)}{U(s)} = \frac{k}{ms^2 + bs + k} = \frac{1.6}{s^2 + 0.5s + 1.6}
$$

In [ ]:
# System parameters
m = 1
b = 0.5
k = 1.6

# Transfer function H(s) = k / (ms^2 + bs + k)
numerator = [k]
denominator = [m, b, k]

# Create transfer function
system = signal.TransferFunction(numerator, denominator)

# Define a larger frequency range for Bode plot
frequencies = np.logspace(-2, 4, 500)  # from 0.01 rad/s to 100 rad/s
w, mag, phase = signal.bode(system, w=frequencies)

# Plot Bode magnitude and phase on a single plot
fig, ax1 = plt.subplots(2, 1, figsize=(10, 8))

# Magnitude plot
ax1[0].semilogx(w, mag)
ax1[0].set_title("Bode Plot")
ax1[0].set_ylabel("Magnitude [dB]")
ax1[0].grid(True)

# Phase plot
ax1[1].semilogx(w, phase)
ax1[1].set_xlabel("Frequency [rad/s]")
ax1[1].set_ylabel("Phase [degrees]")
ax1[1].grid(True)

plt.show()

There are several things we can see in the animation plot and the Bode plot:

- At low frequencies, there is no phase shift and the magnitude is 1.
- At the frequency of roughly $1dB$ the magnitude peaks and the phase rapidly decreases.
- At higher frequencies, there is a phase shift of roughly $180\degree$ and the magnitude goes towards 0.

Credits: https://lpsa.swarthmore.edu